In [1]:
# WEEK 06 HOMEWORK

In [ ]:
### QUESTION 1 ###

In [ ]:
# v22.3.5 (rev 28b2443)

In [ ]:
### QUESTION 2 ###

In [ ]:
# TOPIC       STATUS
# test-topic  OK

In [ ]:
### QUESTION 3 ###

In [1]:
import json
import time

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [ ]:
### QUESTION 4 ###

In [2]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()
print(f'sending message took {(t1 - t0):.2f} seconds')

t2 = time.time()
producer.flush()
t3 = time.time()

print(f'flushing took {(t3 - t2):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
sending message took 0.61 seconds
flushing took 0.00 seconds


In [ ]:
### QUESTION 5 ###

In [3]:
### added this row to unzip gzip to csv
import gzip, shutil

with gzip.open('green_tripdata_2019-10.csv.gz', 'r') as f_in, open('green_tripdata_2019-10.csv', 'wb') as f_out:
  shutil.copyfileobj(f_in, f_out)

In [4]:
import pandas as pd

df = pd.read_csv('green_tripdata_2019-10.csv')

/tmp/ipykernel_149115/2939510799.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('green_tripdata_2019-10.csv')


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               387007 non-null  float64
 1   lpep_pickup_datetime   476386 non-null  object 
 2   lpep_dropoff_datetime  476386 non-null  object 
 3   store_and_fwd_flag     387007 non-null  object 
 4   RatecodeID             387007 non-null  float64
 5   PULocationID           476386 non-null  int64  
 6   DOLocationID           476386 non-null  int64  
 7   passenger_count        387007 non-null  float64
 8   trip_distance          476386 non-null  float64
 9   fare_amount            476386 non-null  float64
 10  extra                  476386 non-null  float64
 11  mta_tax                476386 non-null  float64
 12  tip_amount             476386 non-null  float64
 13  tolls_amount           476386 non-null  float64
 14  ehail_fee              0 non-null   

In [6]:
df_green = df[['lpep_pickup_datetime',
'lpep_dropoff_datetime',
'PULocationID',
'DOLocationID',
'passenger_count',
'trip_distance',
'tip_amount']]

In [7]:
df_green.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [8]:
topic_name = 'green-trips'
t0 = time.time()

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
    #print(f"Sent: {row_dict}")

t1 = time.time()
print(f'sending the messages took {(t1 - t0):.2f} seconds')

sending the messages took 72.92 seconds


In [9]:
producer.flush()

In [ ]:
### QUESTION 6 ###

In [10]:
import pyspark
from pyspark.sql import SparkSession

In [11]:
pyspark_version = pyspark.__version__

In [12]:
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

In [13]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/04/09 11:01:54 WARN Utils: Your hostname, codespaces-4306db resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/04/09 11:01:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/workspaces/de-zoom-by-dimi/data-engineering-zoomcamp-dimi/05-batch/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-25c63d9a-4847-409e-9cb8-7ce66c93ae08;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:

24/04/09 11:01:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [15]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/09 11:02:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0d106d6a-b7b0-4a39-8886-a5483c50d71b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 11:02:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 9, 10, 59, 26, 182000), timestampType=0)


In [16]:
query.stop()

In [17]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [18]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [19]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/04/09 11:02:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9d5dad0a-03d6-434e-b1f5-32c8880d487f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 11:02:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [ ]:
### QUESTION 7 ###

In [20]:
from pyspark.sql.functions import current_timestamp, window, col

In [21]:
green_stream_with_timestamp = green_stream.withColumn("timestamp", current_timestamp())

In [22]:
popular_destinations = green_stream_with_timestamp \
    .groupBy(window(col("timestamp"), "5 minutes"), col("DOLocationID")) \
    .count() \
    .orderBy("count", ascending=False)

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/04/09 11:03:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-849403cc-6ab3-4937-907c-6be006952832. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 11:03:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|74          |17741|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|42          |15942|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|41          |14061|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|75          |12840|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|129         |11930|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|7           |11533|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|166         |10845|
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|236         |7913 |
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|223         |7542 |
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|238         |7318 |
|{2024-04-09 11:00:00, 2024-04-09 11:05:00}|82          |7292 |
|{2024-